# 株価の時系列データの分析

pandas.DataFrame.rolling　https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html  
    

In [ ]:
# -*- coding: utf-8 -*-
# Rolling mean statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

#import matplotlib.finance as mpf
from mpl_finance import candlestick_ohlc
from matplotlib.dates import date2num

matplotlib.rcParams['font.family'] ='Yu Mincho' # matplotlibで日本語を使う

FLAG_fig = False

In [ ]:
url = 'https://sites.google.com/site/datasciencehiro/datasets/Stock_7203.txt'
df = pd.read_csv(url, index_col='日付', parse_dates=[0] )
df = df.sort_index() # 日付を昇順に並び替える
df.head()

#### 移動平均
単純平均と指数重み付き平均

In [ ]:
item='終値'
W = 25
ax = df[item].plot(color='black', label=item)

rol = df[item].rolling(window=W).mean()
rol.plot(ax=ax, label='移動平均25日', color='red', linestyle='dashed')

ewm = df[item].ewm(span=W).mean()
ewm.plot(ax=ax, label='指数移動平均スパン25', color='green', linestyle='dashed')

plt.grid()
plt.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_result_1.png')
plt.show()

#### 移動平均,　windowの大きさの影響
W＝7日と28日

In [ ]:
item='終値'
ax = df[item].plot(color='black', label=item)

rol1 = df[item].rolling(window=7).mean()
rol1.plot(ax=ax, label='移動平均7日', color='red', linestyle='dashed')

rol2 = df[item].rolling(window=28).mean()
rol2.plot(ax=ax, label='移動平均28日', color='green', linestyle='dashed')

plt.grid()
plt.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_result_2.png')
plt.show()

#### ボリンジャーバンド（Bollinger bands)

In [ ]:
item = '終値'
W = 25
ax = df[item].plot(label=item, color='black', linestyle='solid')
rolm = df[item].rolling(window=W).mean()
rolst = df[item].rolling(window=W).std() #unbiased
m = 2
upper_band = rolm + rolst*m
lower_band = rolm - rolst*m

rolm.plot(ax=ax, label='移動平均25日', color='green', linestyle='solid')
upper_band.plot(ax=ax, label='Upper band', color='red', linestyle='dashed')
lower_band.plot(ax=ax, label='Lower band', color='blue', linestyle='dashed')

plt.grid()
plt.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_TSA_StockPrice_BollingerBand.png')
plt.show()

#### ローソク足チャート
・もともと，matplotlib.financeにあったが，現在はmaplotlibは提供していない。  
・現在は，githubから提供：　https://github.com/matplotlib/mpl_finance  
・ただし，使用法は次を参照されたい：https://matplotlib.org/api/finance_api.html

In [ ]:
df0 = df.loc['20170901':'20170930', ['始値','高値','安値','終値']] #2017年9月，1か月だけを見る

df0['終値'].plot(label='9月の終値') #終値のプロット
plt.legend(loc='upper left')
if FLAG_fig: plt.savefig('fig_TSA_StockPrice_Plot_CandleStick_1.png')
    
fig, ax = plt.subplots()

xdate = [x.date() for x in df0.index] #Timestamp -> datetime
ohlc = np.vstack((date2num(xdate), df0.values.T)).T #datetime -> float
candlestick_ohlc(ax, ohlc, width=0.7, colorup='g', colordown='r')

plt.title('9月のローソク足')
ax.grid() #グリッド表示
ax.set_xlim(df0.index[0].date(), df0.index[-1].date()) #x軸の範囲
fig.autofmt_xdate() #x軸のオートフォーマット


if FLAG_fig: plt.savefig('fig_TSA_StockPrice_Plot_CandleStick_2.png')
plt.show()

### 備考　nullデータの対処
・Yafooファナンスから，日経データをダウンロード。
・ファイル名"^N225.csv"，エンコードはutf-8である。
・データにはnullデータ（マーケットが閉じている日は取引が無い）がある<br>
・pandasはnull（データは無いという意味）をNan(Not a Number, 数字データは無いという意味)と解釈する。
・この対処法を示す
・原データを格納する変数（データフレーム）をdf0とする<br>


In [ ]:
url = 'https://sites.google.com/site/datasciencehiro/datasets/^N225.csv'
df0 = pd.read_csv(url, index_col='Date', parse_dates=[0] )
df0 = df0.sort_index() # 日付を昇順に並び替える
df0.head()

#### 原データからNan(nullデータ)を削除して，これを変数dfに代入する
メッソド".notnull()"はNan(null)データが無い行を返す。  
データが無い日は，Open, High, Low, Close　全て同じであるから，ここでは'Open'を用いる 

In [ ]:
df = df0[df0['Open'].notnull()]
df.head()

#### 移動平均

In [ ]:
item='Close'
ax = df[item].plot(color='black', label=item)

rol1 = df[item].rolling(window=5).mean()
rol1.plot(ax=ax, label='Window=5', color='red', linestyle='dashed')

rol2 = df[item].rolling(window=25).mean()
rol2.plot(ax=ax, label='Window=25', color='green', linestyle='dashed')


plt.grid()
plt.legend(loc='upper right')
plt.show()